In [93]:
import pandas as pd
import numpy as np
from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error,v_measure_score
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
import matplotlib as plt
%matplotlib inline
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE

In [123]:
ratings_df = pd.read_csv('/home/anushya/Documents/ml-100k/u.data',sep="\t",header=None)
ratings_df.columns = ['user_id' ,'MovieID', 'rating','timestamp']
ratings_df

,user_id,MovieID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [125]:
user_count = ratings_df['user_id'].unique().shape[0]
items_count = ratings_df['MovieID'].unique().shape[0]
print (user_count,items_count)

943 1682


In [126]:
group_user = ratings_df.groupby('user_id')['rating'].count()
pd.DataFrame(group_user.sort_values(ascending = False))

,rating
user_id,
405,737
655,685
13,636
450,540
276,518
416,493
537,490
303,484
234,480


# Split the data using crossvalidation

In [127]:
 train_data, test_data = cv.train_test_split(ratings_df, test_size = 0.25)

# Creating Matrix from Dataframe

In [128]:
train_matrix = np.zeros((user_count, items_count))
for line in train_data.itertuples():
    train_matrix[line[1]-1, line[2]-1] = line[3]
test_matrix = np.zeros((user_count, items_count))
for line in test_data.itertuples():
    test_matrix[line[1]-1, line[2]-1] = line[3]
data_df = pd.DataFrame(train_matrix)
data_df.reset_index()



,index,0,1,2,3,4,5,6,7,8,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0,5.0,0.0,4.0,0.0,0.0,5.0,4.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
user_similarity = 1-(pairwise_distances(train_matrix, metric='cosine'))
item_similarity = 1-(pairwise_distances(train_matrix.T, metric='cosine'))
df = pd.DataFrame(user_similarity)
#df[3]

In [130]:
k = df[3].sort_values(ascending = False).head(10)
k

3      1.000000
569    0.389819
817    0.370336
32     0.356016
354    0.319426
450    0.279958
2      0.277933
819    0.274582
104    0.274172
549    0.273724
Name: 3, dtype: float64

In [131]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred


In [132]:
item_prediction = predict(train_matrix, item_similarity, type='item')
user_prediction = predict(train_matrix, user_similarity, type='user')
df1 = pd.DataFrame(user_prediction)
df1
#user_prediction

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.094619,0.731233,0.485619,1.268127,0.540078,0.300718,1.753152,1.202320,1.434905,0.553103,...,0.186528,0.183363,0.186883,0.185710,0.189337,0.183753,0.183363,0.184143,0.183363,0.190751
1,1.545320,0.179408,0.150702,0.490891,0.123781,0.027415,1.139999,0.514123,1.013084,0.240120,...,-0.053958,-0.056387,-0.053299,-0.054328,-0.051431,-0.055077,-0.056387,-0.053767,-0.056387,-0.054584
2,0.947387,0.137412,0.101011,0.353838,0.109359,-0.000886,0.849663,0.333370,0.659158,0.154584,...,-0.059842,-0.060409,-0.054113,-0.056211,-0.057557,-0.056192,-0.060409,-0.051976,-0.060409,-0.059249
3,1.168394,0.128792,0.067454,0.403829,0.093922,-0.058499,0.959655,0.379473,0.705988,0.108415,...,-0.106694,-0.109390,-0.104988,-0.106455,-0.104166,-0.106736,-0.109390,-0.104082,-0.109390,-0.108320
4,2.081177,0.663232,0.268074,1.169993,0.390073,0.057480,1.583981,1.066211,1.087509,0.325477,...,-0.018961,-0.022235,-0.018718,-0.019890,-0.017702,-0.022235,-0.022235,-0.022235,-0.022235,-0.014782
5,1.851670,0.536594,0.335234,1.052250,0.380334,0.218037,1.508653,1.076477,1.373201,0.499128,...,0.112694,0.108259,0.111443,0.110382,0.115274,0.108771,0.108259,0.109283,0.108259,0.113757
6,2.275827,1.045747,0.727738,1.631616,0.840864,0.559199,1.994110,1.581584,1.703529,0.864915,...,0.470978,0.465660,0.468545,0.467583,0.471593,0.465893,0.465660,0.466125,0.465660,0.472692
7,1.688271,0.450206,0.129575,0.916881,0.217059,-0.046183,1.433056,0.771846,0.916996,0.189353,...,-0.107744,-0.110191,-0.105788,-0.107255,-0.106179,-0.109852,-0.110191,-0.109513,-0.110191,-0.104387
8,1.364700,0.179924,0.069250,0.521983,0.087366,-0.047616,1.236725,0.558189,0.914408,0.185287,...,-0.125989,-0.127758,-0.125712,-0.126394,-0.122773,-0.125894,-0.127758,-0.124029,-0.127758,-0.123305
9,1.842395,0.556883,0.331156,1.151586,0.396361,0.198683,1.498224,1.119319,1.408300,0.477870,...,0.091585,0.086545,0.089164,0.088291,0.093319,0.086851,0.086545,0.087156,0.086545,0.092633


In [133]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [134]:
rmse(item_prediction, test_matrix)

3.173882232411968

In [135]:
rmse(user_prediction, test_matrix)

2.969342676146433

In [136]:


knn = KNeighborsRegressor(n_neighbors=5)

# Recommendation

In [137]:
def predict_topk(ratings, similarity, kind='user', k=20):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        
    
    return pred

In [138]:
pred = predict_topk(train_matrix, user_similarity, kind='user', k=20)
pred1 = predict_topk(train_matrix, item_similarity, kind='item', k=20)




In [139]:
pred

array([[ 4.03421945,  1.1683438 ,  1.64315952, ...,  0.        ,
         0.        ,  0.14768185],
       [ 2.3018829 ,  0.        ,  0.26522927, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.09442056,
         0.        ,  0.        ],
       ..., 
       [ 3.32205631,  0.        ,  0.38182982, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.05792985,  0.44725138,  0.08251435, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.06073486,  1.41517318,  0.73234036, ...,  0.        ,
         0.        ,  0.        ]])

In [140]:
pred1

array([[ 2.79858461,  1.13071314,  1.95227795, ...,  0.        ,
         0.        ,  0.27437588],
       [ 1.45674926,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.2134894 ,  0.        ,  0.        , ...,  0.07699932,
         0.        ,  0.        ],
       ..., 
       [ 0.8601204 ,  0.        ,  0.32907181, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.91153415,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.16885048,  2.30319251,  0.95296617, ...,  0.        ,
         0.        ,  0.        ]])

creating a table of user_id,item_id

In [141]:
R_df = ratings_df.pivot(index ='user_id' , columns = 'MovieID',values = 'rating').fillna(0)
R_df.head()


MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## converting df to input matrix for svd

In [142]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demean = R - user_ratings_mean.reshape(-1, 1)
R_demean

array([[ 4.41617122,  2.41617122,  3.41617122, ..., -0.58382878,
        -0.58382878, -0.58382878],
       [ 3.86325803, -0.13674197, -0.13674197, ..., -0.13674197,
        -0.13674197, -0.13674197],
       [-0.08977408, -0.08977408, -0.08977408, ..., -0.08977408,
        -0.08977408, -0.08977408],
       ..., 
       [ 4.9470868 , -0.0529132 , -0.0529132 , ..., -0.0529132 ,
        -0.0529132 , -0.0529132 ],
       [-0.20035672, -0.20035672, -0.20035672, ..., -0.20035672,
        -0.20035672, -0.20035672],
       [-0.34066587,  4.65933413, -0.34066587, ..., -0.34066587,
        -0.34066587, -0.34066587]])

svd

In [143]:
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)


prediction

In [144]:
user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
pred_df = pd.DataFrame(user_predicted_ratings,columns = R_df.columns)
pred_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


In [145]:
item_data = pd.read_csv('/home/anushya/Documents/ml-100k/u.item',encoding='latin-1',sep = '|',header=None)
item_data.columns = ['movie id','movie title', 'release date' ,'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Children' , 'Comedy' ,'Crime' , 'Documentary ', 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' ,'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' ,' Western']
#item_data.head()

In [146]:
movies_list = [i.strip().split("::") for i in open('/home/anushya/Documents/ml-100k/movies.dat', 'r', encoding='latin-1').readlines()]
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [171]:


def recommend_movies(predictions_df, userID, movies_df, ratings_df, num_recommendations=5):
    user_row_number = userID - 1
    sorted_user_predictions = pred_df.iloc[user_row_number].sort_values(ascending=False) 
    user_data = ratings_df[ratings_df.user_id == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').sort_values(['rating'], ascending=False))

   # print (format(userID, user_full.shape[0]))
    #print (format(num_recommendations))
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    return user_full, recommendations



In [172]:
sorted_user_predictions = pred_df.iloc[500].sort_values(ascending=False) 
#sorted_user_predictions
#pd.DataFrame(sorted_user_predictions).reset_index()

In [173]:
already_rated, predictions = recommend_movies(pred_df, 400, movies_df, ratings_df, 10)

In [174]:
already_rated

,user_id,MovieID,rating,timestamp,Title,Genres
21,400,313,5,885676316,"Swan Princess, The (1994)",Animation|Children's
17,400,258,5,885676316,"Kid in King Arthur's Court, A (1995)",Adventure|Children's|Comedy|Fantasy|Romance
9,400,323,4,885676582,NaN,NaN
20,400,343,4,885676552,"Baby-Sitters Club, The (1995)",Children's
18,400,269,4,885676230,My Crazy Life (Mi vida loca) (1993),Drama
14,400,301,4,885676411,Picture Bride (1995),Drama|Romance
1,400,300,4,885676230,Quiz Show (1994),Drama
10,400,288,4,885676365,Natural Born Killers (1994),Action|Thriller
11,400,286,4,885676230,Nemesis 2: Nebula (1995),Action|Sci-Fi|Thriller
7,400,749,4,885676452,"Man from Down Under, The (1943)",Drama


In [175]:
predictions

,MovieID,Title,Genres
314,333,Tommy Boy (1995),Comedy
309,326,To Live (Huozhe) (1994),Drama
291,302,Queen Margot (La Reine Margot) (1994),Drama|Romance
655,678,Some Folks Call It a Sling Blade (1993),Drama|Thriller
242,245,"Glass Shield, The (1994)",Crime|Drama
281,289,Only You (1994),Comedy|Romance
306,322,Swimming with Sharks (1995),Comedy|Drama
296,310,Rent-a-Kid (1995),Comedy
266,272,"Madness of King George, The (1994)",Drama
284,292,Outbreak (1995),Action|Drama|Thriller
